In [ ]:
username = ""
source = "";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

In [ ]:
Random.seed!(hash(username))
abtest_order = Random.shuffle(1:3);

# Variant A

In [ ]:
recommendations(abtest_order[1])

# Variant B

In [ ]:
recommendations(abtest_order[2])

# Variant C

In [ ]:
recommendations(abtest_order[3])